In [1]:
#pip install keras==2.12.0


In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import holidays

from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV, GridSearchCV, TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [6]:
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer

In [7]:
#Set parameters to see all data
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [8]:
# Read dataset

%%time
data_mart_path = 'data_mart.csv'

try:
    data_mart_df = pd.read_csv(data_mart_path)
except pd.errors.ParserError as e:
    print(f'Error while parsing CSV file: {e}')



data_mart_df.shape

<timed exec>:4: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 1.97 s, sys: 333 ms, total: 2.3 s
Wall time: 4.37 s


(101350, 104)

In [9]:
# Method for reducing the memory usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [10]:
data_mart_df = reduce_mem_usage(data_mart_df)

Mem. usage decreased to 37.89 Mb (52.9% reduction)


In [11]:
# Replace empty spaces of dataframe column names with  '_'
data_mart_df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
# Replace uppercase characters of dataframe column names with lowercase
data_mart_df.columns = data_mart_df.columns.str.lower()

In [12]:
# Create a dictionary to merge similar values
company_type_merge_dict = {
    'Limited (LTD)': 'Limited',
    'Partnership – 3 or less': 'Partnership',
    'Partnership – 4 or more': 'Partnership',
}

# Merge similar values using the dictionary
data_mart_df['company_type'] = data_mart_df['company_type'].replace(company_type_merge_dict)

# Display unique values
print(data_mart_df['company_type'].unique())


[nan 'Limited' 'Other' 'Public Limited Company (PLC)' 'Sole Trader'
 'Partnership' 'Council/Charity']


In [13]:
companycategory_merge_dict = {
    'PRI/LTD BY GUAR/NSC (Private, limited by guarantee, no share capital)': 'Private Limited Company',
    "PRI/LBG/NSC (Private, Limited by guarantee, no share capital, use of 'Limited' exemption)": 'Private Limited Company',
    'PRIV LTD SECT. 30 (Private limited company, section 30 of the Companies Act)': 'Private Limited Company'
}

# Merge similar values using the dictionary
data_mart_df['companycategory'] = data_mart_df['companycategory'].replace(companycategory_merge_dict)
# Display unique values
print(data_mart_df['company_type'].unique())


[nan 'Limited' 'Other' 'Public Limited Company (PLC)' 'Sole Trader'
 'Partnership' 'Council/Charity']


In [14]:
# Creating a dictionary to merge similar values
uk_region_merge_dict = {
    'East of England': 'East England',
    'East Midlands': 'East England',
}

# Merging similar values using the dictionary
data_mart_df['uk_region'] = data_mart_df['uk_region'].replace(uk_region_merge_dict)

In [15]:
# Creating a dictionary to merge similar values and shorten names
ruc11_merge_dict = {
    '3_Urban with Significant Rural (rural including hub towns 26-49%)': 'Urban with Significant Rural',
    '2_Largely Rural (rural including hub towns 50-79%) ': 'Largely Rural',
    '6_Urban with Major Conurbation': 'Urban with Major Conurbation',
    '4_Urban with City and Town': 'Urban with City and Town'
}

# Merging similar values and shortening names using the dictionary
data_mart_df['ruc11'] = data_mart_df['ruc11'].replace(ruc11_merge_dict)

In [16]:
# Check for missing values
missing_values = data_mart_df.isnull().sum()
missing_percentage = (missing_values / len(data_mart_df)) * 100
print(pd.concat([missing_values, missing_percentage], axis=1, keys=['Missing Values', 'Percentage']))

                                                    Missing Values  Percentage
toplevelcustomerid                                               0    0.000000
client_account_number                                            0    0.000000
credit_terms_requested                                       44300   43.709916
credit_terms_granted                                         61021   60.208189
company_type                                                 60666   59.857918
cash_flow_score                                              57022   56.262457
credit_limit_requested                                        6216    6.133202
credit_limit_assigned                                        19843   19.578688
num_employees                                                10569   10.428219
fleet_size_declared                                          58334   57.556981
account_source                                                3830    3.778984
lead_type                                           

In [17]:
# Create a list of columns with missing_percentage greater than 60% using the above code
high_missing_columns = missing_percentage[missing_percentage > 60].index.tolist()

# Display highly missing columns
print(len(high_missing_columns))
print(high_missing_columns)

32
['credit_terms_granted', 'lead_type', 'avg_retention_cases_per_year', 'avg_support_cases_per_year', 'avg_complaint_cases_per_year', 'promo_group', 'average_director_tenure', 'customertype', 'survey_year', 'survey_product_satisfaction', 'survey_support_satisfaction', 'survey_product_issue_last_6_months', 'survey_documentation_satisfaction', 'survey_valued_customer', 'survey_fuelcard_plus_expenses', 'survey_fleet_management', 'survey_national_fuel_network', 'survey_ev_payments', 'survey_fuel_discounts', 'survey_breakdown_cover', 'survey_parking', 'survey_online_account', 'survey_continue_using_product', 'survey_value_for_money', 'survey_comparison_to_other_companies', 'survey_hybrid_fleet_percentage', 'survey_ev_fleet_percentage', 'num_no_class_used', 'num_cars_used', 'num_lgv_used', 'num_hgv_used', 'num_total_vehicles_used']


In [18]:
data_mart_df.shape

(101350, 104)

In [19]:
# Drop negative values
data_mart_df = data_mart_df[(data_mart_df['total_spend_monthly_avg'] >= 0) &
                            (data_mart_df['total_transactions_monthly_avg'] >= 0) &
                            (data_mart_df['total_rev_monthly_avg'] >= 0)]

In [20]:
data_mart_df.shape

(79066, 104)

In [21]:
# Check for missing values
CLV_missing_values = data_mart_df[['total_spend_monthly_avg', 'total_transactions_monthly_avg',
                               'total_rev_monthly_avg', 'churned']].isnull().sum()
CLV_missing_percentage = (CLV_missing_values / len(data_mart_df)) * 100
print(pd.concat([CLV_missing_values, CLV_missing_percentage], axis=1, keys=['Missing Values', 'Percentage']))

                                Missing Values  Percentage
total_spend_monthly_avg                      0         0.0
total_transactions_monthly_avg               0         0.0
total_rev_monthly_avg                        0         0.0
churned                                      0         0.0


In [22]:
data_mart_df_V2 = data_mart_df.copy()
data_mart_df_V2.shape

(79066, 104)

In [23]:
# Drop columns from the DataFrame
data_mart_df_V2.drop(columns=high_missing_columns, inplace=True)
print(data_mart_df_V2.shape)


(79066, 72)


In [24]:
cat_col_list = []
for col in data_mart_df_V2.select_dtypes(include=['object']).columns:
  #print(col, " : \n")
  if (data_mart_df[col].nunique() < 15):
    cat_col_list.append(col)
print(cat_col_list)

['company_type', 'domain_type', 'lineofbusiness', 'companycategory', 'uk_region', 'country', 'ruc11']


In [25]:
data_mart_df_V2['start_date'] = pd.to_datetime(data_mart_df_V2['start_date'], format='%d/%m/%Y')

data_mart_df_V2.start_date.min(), data_mart_df_V2.start_date.max()

(Timestamp('1990-08-13 00:00:00'), Timestamp('2024-03-04 00:00:00'))

In [26]:
data_mart_df_V2['last_transaction_date'] = pd.to_datetime(data_mart_df_V2['last_transaction_date'], format='%d/%m/%Y')

data_mart_df_V2.last_transaction_date.min(), data_mart_df_V2.last_transaction_date.max()

(Timestamp('2019-10-06 00:00:00'), Timestamp('2024-03-15 00:00:00'))

In [27]:
# Date of The First Covid-19 Case and Brexit in the UK
pandemic_date = '31/01/2020'
brexit_date = '23/06/2016'

In [28]:
# Check data size of (pandemic) threshold days
print('Before 2000 => ', data_mart_df_V2[data_mart_df_V2.start_date < '01/01/2000'].shape[0])
print('Between 2000 and 2010 => ', data_mart_df_V2[(data_mart_df_V2.start_date >= '01/01/2000') & (data_mart_df_V2.start_date < '01/01/2010')].shape[0])
print('Between 2000 and Pandemic => ', data_mart_df_V2[(data_mart_df_V2.start_date >= '01/01/2000') & (data_mart_df_V2.start_date < pandemic_date)].shape[0])
print('After Pandemic => ', data_mart_df_V2[data_mart_df_V2.start_date >= pandemic_date].shape[0])


Before 2000 =>  3722
Between 2000 and 2010 =>  9989
Between 2000 and Pandemic =>  33731
After Pandemic =>  41613


In [29]:
# Check data size of (Brexit) threshold days
print('Before 2000 => ', data_mart_df_V2[data_mart_df_V2.start_date < '01/01/2000'].shape[0])
print('Between 2000 and 2010 => ', data_mart_df_V2[(data_mart_df_V2.start_date >= '01/01/2000') & (data_mart_df_V2.start_date < '01/01/2010')].shape[0])
print('Between 2000 and Brexit => ', data_mart_df_V2[(data_mart_df_V2.start_date >= '01/01/2000') & (data_mart_df_V2.start_date < brexit_date)].shape[0])
print('After Brexit => ', data_mart_df_V2[data_mart_df_V2.start_date >= brexit_date].shape[0])

Before 2000 =>  3722
Between 2000 and 2010 =>  9989
Between 2000 and Brexit =>  20413
After Brexit =>  54931


In [30]:
data_mart_df_V3 = data_mart_df_V2.copy()
data_mart_df_V3.shape

(79066, 72)

In [31]:
# Check for missing values
CLV_missing_date_values = data_mart_df_V3[['start_date', 'last_transaction_date']].isnull().sum()
CLV_date_missing_percentage = (CLV_missing_date_values / len(data_mart_df_V3)) * 100
print(pd.concat([CLV_missing_date_values, CLV_date_missing_percentage], axis=1, keys=['Missing Values', 'Percentage']))

                       Missing Values  Percentage
start_date                          0         0.0
last_transaction_date               0         0.0


In [32]:
data_mart_df_V3[['start_date', 'last_transaction_date']].head()

,start_date,last_transaction_date
0,2001-02-01,2024-02-02
1,2001-02-06,2024-02-28
2,2001-02-06,2024-02-12
3,2001-02-06,2024-02-22
4,2004-02-15,2024-02-26


In [33]:
data_mart_df_V3[data_mart_df_V3.last_transaction_date < '2024-03-09'].shape, data_mart_df_V3[data_mart_df_V3.last_transaction_date >= '2024-03-09'].shape

((61092, 72), (17974, 72))

In [34]:
# Load UK holidays
uk_holidays = holidays.UnitedKingdom()

# Function to check if a date is a holiday
def is_holiday(date_value):
    return date_value in uk_holidays

In [35]:
# Apply the function to create a holiday flag column
data_mart_df_V3['is_holiday'] = data_mart_df_V3['start_date'].apply(is_holiday).astype(int)

In [36]:
# Customer active days
data_mart_df_V3['customer_activity_duration'] = (data_mart_df_V3['last_transaction_date'] - data_mart_df_V3['start_date']).dt.days

In [37]:
data_mart_df_V3.customer_activity_duration.mean()

2298.8099309437684

In [38]:
# Calculating day difference for date column
def calculate_day_difference(df, column_name):
    df[column_name] = pd.to_datetime(df[column_name], format='%d/%m/%Y')#.dt.date
    df[f"{column_name}_diff"] = (df['last_transaction_date'].max() - df[column_name]).dt.days
    return df

In [39]:
data_mart_df_V3 = calculate_day_difference(data_mart_df_V3, 'start_date')
data_mart_df_V3 = calculate_day_difference(data_mart_df_V3, 'last_transaction_date')

data_mart_df_V3[['start_date', 'start_date_diff', 'last_transaction_date', 'last_transaction_date_diff']].head()

,start_date,start_date_diff,last_transaction_date,last_transaction_date_diff
0,2001-02-01,8443,2024-02-02,42
1,2001-02-06,8438,2024-02-28,16
2,2001-02-06,8438,2024-02-12,32
3,2001-02-06,8438,2024-02-22,22
4,2004-02-15,7334,2024-02-26,18


In [40]:
# Calculate churn rate
churn_rate = data_mart_df_V3['churned'].value_counts(normalize=True) * 100
print("Churn Rate:\n", churn_rate)

Churn Rate:
 churned
0    55.690183
1    44.309817
Name: proportion, dtype: float64


In [41]:
# Customer Diversity
data_mart_df_V3['transaction_diversity'] = data_mart_df_V3[['fuel_usage_flag', 'ev_usage_flag', 'rev_toll_usage_flag']].sum(axis=1)
data_mart_df_V3[['transaction_diversity', 'fuel_usage_flag', 'ev_usage_flag', 'rev_toll_usage_flag']].head()


,transaction_diversity,fuel_usage_flag,ev_usage_flag,rev_toll_usage_flag
0,1,1,0,0
1,1,1,0,0
2,1,1,0,0
3,1,1,0,0
4,1,1,0,0


In [42]:
data_mart_df_V3.iloc[:, :35].head()

,toplevelcustomerid,client_account_number,credit_terms_requested,company_type,cash_flow_score,credit_limit_requested,credit_limit_assigned,num_employees,fleet_size_declared,account_source,domain_type,start_date,num_live_cards,fuel_usage_flag,ev_usage_flag,rev_toll_usage_flag,servicepointepyx_usage_flag,kwikfit_usage_flag,bmm_usage_flag,beyondfuel_usage_flag,commercialdelphicreditlimit,commercialdelphiscore,numberofccjsinlast2years,lineofbusiness,last_transaction_date,tenure,fuel_active,visa_active,ev_active,fuel_sites_monthly_avg,total_fee_charge_monthly_avg,number_of_distinct_fees_monthly_avg,number_of_web_users,number_of_logins_monthly_avg,siccode_sictext_1
0,100000,LAC29892001,NaN,NaN,NaN,0.0,NaN,NaN,NaN,3rd Party - Murco Local Account,personal,2001-02-01,2.0,1,0,0,0,0,0,0,1083.333374,75.3125,0.0,AllStar,2024-02-02,8400.0,1.0,0.0,0.0,1.0,17.0,2.0,NaN,NaN,NaN
1,100003,LAC29948001,NaN,NaN,NaN,0.0,NaN,3.0,NaN,3rd Party - Murco Local Account,commercial,2001-02-06,5.0,1,0,0,0,0,0,0,1250.000000,82.0000,0.0,AllStar,2024-02-28,8424.0,1.0,0.0,0.0,1.0,13.0,1.0,1.0,2.0,NaN
2,100004,LAC29949001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3rd Party - Murco Local Account,personal,2001-02-06,2.0,1,0,0,0,0,0,0,NaN,NaN,NaN,AllStar,2024-02-12,8408.0,1.0,0.0,0.0,1.0,5.0,1.0,1.0,2.0,NaN
3,100005,LAC29950001,NaN,NaN,NaN,0.0,NaN,1.0,NaN,3rd Party - Murco Local Account,personal,2001-02-06,12.0,1,0,0,0,0,0,0,906.250000,54.1250,0.0,AllStar,2024-02-22,8416.0,1.0,0.0,0.0,1.0,38.0,0.0,1.0,3.0,NaN
4,10002,28709954,NaN,NaN,NaN,0.0,NaN,4.0,NaN,Visa Upsell,personal,2004-02-15,1.0,1,0,0,0,0,0,0,8757.142578,83.3125,0.0,AllStar,2024-02-26,7316.0,1.0,0.0,0.0,1.0,27.0,4.0,1.0,2.0,NaN


In [43]:
# Custom encoding for 'credit_terms_requested'
def custom_encoding(term):
    if pd.isna(term):
        return 100  # Return 100 for NaN values

    # Replace date words with appropriate integers
    term = term.replace('Monthly', '30')
    term = term.replace('Weekly', '7')
    term = term.replace('Fortnightly', '14')
    term = term.replace('Others', '100')
    term = term.replace('Other', '100')
    term = term.replace('DA', '1')
    term = term.replace('Best Available', '0')

    # Split terms and calculate total days
    terms = term.split('+')
    days = sum(int(x) for x in terms)

    return days

In [44]:
data_mart_df_V3['credit_terms_requested'] = data_mart_df_V3['credit_terms_requested'].apply(custom_encoding)
data_mart_df_V3['credit_terms_requested'].unique()

array([100,  14,  21,  37,  28,  42,  35,  44,  67,   0,  60,  24,  45,
        58,  51,  17,   1,  40,  12])

In [45]:
# Replace NaN with 'Other'
data_mart_df_V3['company_type'] = data_mart_df_V3['company_type'].fillna('Other')

# Replace white space with underscores
data_mart_df_V3['company_type'] = data_mart_df_V3['company_type'].str.replace(' ', '_')
data_mart_df_V3['company_type'] = data_mart_df_V3['company_type'].str.replace('/', '_')

# Replace 'Public Limited Company (PLC)' with 'PLC'
data_mart_df_V3['company_type'] = data_mart_df_V3['company_type'].replace('Public_Limited_Company_(PLC)', 'PLC')

# Display unique values after transformations
print(data_mart_df_V3['company_type'].unique())

['Other' 'Limited' 'PLC' 'Sole_Trader' 'Partnership' 'Council_Charity']


In [46]:
# Convert categorical variable into dummy/indicator variables
data_mart_df_V3 = pd.get_dummies(data_mart_df_V3, columns=['company_type'])

In [47]:
# Get columns containing 'life'
columns_containing_life = [col for col in data_mart_df_V3.columns if 'company_type' in col]
columns_containing_life

['company_type_Council_Charity',
 'company_type_Limited',
 'company_type_Other',
 'company_type_PLC',
 'company_type_Partnership',
 'company_type_Sole_Trader']

In [48]:
data_mart_df_V3.cash_flow_score.min(), data_mart_df_V3.cash_flow_score.mean(), data_mart_df_V3.cash_flow_score.max()

(0.0, inf, 400.0)

In [49]:
data_mart_df_V3 = reduce_mem_usage(data_mart_df_V3)

Mem. usage decreased to 22.09 Mb (11.7% reduction)


In [50]:
# Method for fill nan values with mean value of feature
def fill_nan_values_with_mean(df, feature):
  mean_score = round(df[feature].dropna().astype(np.int32).mean())
  df[feature] = df[feature].fillna(mean_score)
  return df

In [51]:
data_mart_df_V3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79066 entries, 0 to 101349
Data columns (total 82 columns):
 #   Column                                                                                    Non-Null Count  Dtype         
---  ------                                                                                    --------------  -----         
 0   toplevelcustomerid                                                                        79066 non-null  int32         
 1   client_account_number                                                                     79066 non-null  object        
 2   credit_terms_requested                                                                    79066 non-null  int8          
 3   cash_flow_score                                                                           38194 non-null  float16       
 4   credit_limit_requested                                                                    76355 non-null  float32       
 5   credit_l

In [52]:
for col in data_mart_df_V3.select_dtypes(exclude=['object', 'bool', 'datetime64']).columns:
  print(col)
  data_mart_df_V3 = fill_nan_values_with_mean(data_mart_df_V3, col)


toplevelcustomerid
credit_terms_requested
cash_flow_score
credit_limit_requested
credit_limit_assigned
num_employees
fleet_size_declared
num_live_cards
fuel_usage_flag
ev_usage_flag
rev_toll_usage_flag
servicepointepyx_usage_flag
kwikfit_usage_flag
bmm_usage_flag
beyondfuel_usage_flag
commercialdelphicreditlimit
commercialdelphiscore
numberofccjsinlast2years
tenure
fuel_active
visa_active
ev_active
fuel_sites_monthly_avg
total_fee_charge_monthly_avg
number_of_distinct_fees_monthly_avg
number_of_web_users
number_of_logins_monthly_avg
mortgages_nummortcharges
age
sic_code
total_directors_over_time
average_age_of_directors
pence_per_litre
total_population_2011
rural_including_hub_towns_rural_&_rural_related_population_as_%_of_total_population_2011
ruc11cd
tenure_months
total_litres_monthly_avg
non_dd_litres_monthly_avg
dd_litres_monthly_avg
total_transactions_monthly_avg
total_transactions_fuel_monthly_avg
total_transactions_non_visa_monthly_avg
total_transactions_visa_monthly_avg
total_s

In [53]:
data_mart_df_V3.iloc[:, 35:].head()

,age,companycategory,sic_code,section,division,total_directors_over_time,average_age_of_directors,nationality,pence_per_litre,postcode_prefix,postcode_district,uk_region,country,county,total_population_2011,rural_including_hub_towns_rural_&_rural_related_population_as_%_of_total_population_2011,ruc11cd,ruc11,tenure_months,total_litres_monthly_avg,non_dd_litres_monthly_avg,dd_litres_monthly_avg,total_transactions_monthly_avg,total_transactions_fuel_monthly_avg,total_transactions_non_visa_monthly_avg,total_transactions_visa_monthly_avg,total_spend_monthly_avg,fuel_spend_monthly_avg,non_fuel_spend_non_visa_monthly_avg,non_fuel_spend_visa_monthly_avg,total_ev_transactions_monthly_avg,total_ev_energy_charge_per_kwh_monthly_avg,total_ev_session_charge_per_minute_monthly_avg,ev_spend_monthly_avg,total_rev_monthly_avg,churned,is_holiday,customer_activity_duration,start_date_diff,last_transaction_date_diff,transaction_diversity,company_type_Council_Charity,company_type_Limited,company_type_Other,company_type_PLC,company_type_Partnership,company_type_Sole_Trader
0,14.0,NaN,53230.0,NaN,NaN,4.0,50.0,NaN,0.099854,NaN,NaN,NaN,NaN,NaN,970928.0,47.0,3.0,NaN,277.0,38.0,38.0,0.0,1.0,1.0,0.0,0.0,55.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0,0,8401,8443,42,1,False,False,True,False,False,False
1,14.0,NaN,53230.0,NaN,NaN,4.0,50.0,NaN,0.047058,NaN,NaN,NaN,NaN,NaN,970928.0,47.0,3.0,NaN,278.0,66.0,66.0,0.0,2.0,2.0,0.0,0.0,92.0,92.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,0,8422,8438,16,1,False,False,True,False,False,False
2,14.0,NaN,53230.0,NaN,NaN,4.0,50.0,NaN,0.132080,NaN,NaN,NaN,NaN,NaN,970928.0,47.0,3.0,NaN,277.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,8406,8438,32,1,False,False,True,False,False,False
3,14.0,NaN,53230.0,NaN,NaN,4.0,50.0,NaN,0.025986,NaN,NaN,NaN,NaN,NaN,970928.0,47.0,3.0,NaN,278.0,164.0,164.0,0.0,3.0,3.0,0.0,0.0,223.0,222.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0,0,8416,8438,22,1,False,False,True,False,False,False
4,14.0,NaN,53230.0,NaN,NaN,4.0,50.0,NaN,0.135010,NaN,NaN,NaN,NaN,NaN,970928.0,47.0,3.0,NaN,242.0,62.0,62.0,0.0,1.0,1.0,0.0,0.0,87.0,87.0,0.0,0.0,0.0,0.0,0.0,0.0,8,0,0,7316,7334,18,1,False,False,True,False,False,False


In [54]:
def replace_missing_with_other(data):
    # List of NaN, 'Unknown', and other possible missing values
    missing_values = [np.nan, 'Unknown', '0', 'Others', 'NaN', '']

    # Replace missing values with 'Other'
    data.replace(missing_values, 'Other', inplace=True)

    return data
for col in data_mart_df_V3.select_dtypes(include=['object']).columns:
  print(col)
  data_mart_df_V3[col] = replace_missing_with_other(data_mart_df_V3[col])

client_account_number
account_source
domain_type
lineofbusiness
siccode_sictext_1
companycategory
section
division
nationality
postcode_prefix
postcode_district
uk_region
country
county
ruc11


In [55]:
data_mart_df_V3.account_source.unique()

array(['3rd Party - Murco Local Account', 'Visa Upsell', 'Other',
       'Direct - Track Compare', 'Winback', 'Direct - Personal Lead',
       'Data - ReD Direct Marketing', 'Visa - expcomp',
       'Data - Experian Pre Approved', 'Winback AS', '3rd Party - Murco',
       '3rd Party - Other Leasing Partners', '3rd Party - Snax 24',
       'Data - MIB', 'Allstar - Upsell', 'Lapsed - Live Cards',
       'Direct - ASUK', '3rdParty - AA', 'Direct - Email',
       '3rd Party - Arval', '3rd Party - Castle', 'Direct - CFCUK',
       'Direct - Refer-a-Friend', 'Direct - SFCUK', 'Data - Data Captain',
       'Direct - Companeo', 'Direct - Arval', 'Corp TM Referral',
       '3rd Party - HKS', 'Partner - FSB', 'Partner - Quota',
       'Allstar - Network Service Fee Referral', 'Direct - FCUK',
       'Direct - BFCUK', 'Direct - Garage Referral', 'Partner',
       'Data - Dead Source', 'Partner - Keyfuels Lead', 'Visa - expAP',
       'Direct - Approved Index', 'Data - 3DMG',
       'Direct - Dire

In [56]:
data_mart_df_V3['companycategory'] = data_mart_df_V3['companycategory'].str.replace(' ', '_')
data_mart_df_V3.companycategory.unique()

array(['Other', 'Private_Limited_Company', 'Public_Limited_Company',
       'Private_Unlimited_Company', 'Community_Interest_Company'],
      dtype=object)

In [57]:
data_mart_df_V3['uk_region'] = data_mart_df_V3['uk_region'].str.replace(' ', '_')
data_mart_df_V3.uk_region.unique()

array(['Other', 'North_West', 'South_West', 'East_England',
       'West_Midlands', 'South_East', 'Greater_London', 'Wales',
       'North_East', 'Scotland', 'Northern_Ireland', 'Non-geographic',
       'Isle_of_Man'], dtype=object)

In [58]:
data_mart_df_V3['country'] = data_mart_df_V3['country'].str.replace(r' ', '_')

data_mart_df_V3.country.unique()

array(['Other', 'England', 'Wales', 'Scotland', 'Isle_of_Man'],
      dtype=object)

In [59]:
data_mart_df_V3['county'] = data_mart_df_V3['county'].str.replace(r' ', '_')

data_mart_df_V3.county.unique()

array(['Other', 'oxfordshire', 'worcestershire', 'warwickshire',
       'somerset', 'west_sussex', 'hampshire', 'essex', 'hertfordshire',
       'suffolk', 'kent', 'surrey', 'north_yorkshire', 'staffordshire',
       'lancashire', 'nottinghamshire', 'norfolk', 'cumbria',
       'gloucestershire', 'devon'], dtype=object)

In [60]:
data_mart_df_V3['ruc11'] = data_mart_df_V3['ruc11'].str.replace(r' ', '_')

data_mart_df_V3.ruc11.unique()

array(['Other', 'Urban_with_Significant_Rural', 'Largely_Rural',
       'Urban_with_Major_Conurbation', 'Urban_with_City_and_Town'],
      dtype=object)

In [61]:
# Remove leading or trailing spaces and other non-alphabetic characters
data_mart_df_V3['postcode_prefix'] = data_mart_df_V3['postcode_prefix'].str.replace(r'[^a-zA-Z]', '')

# Define a dictionary mapping similar values to a standardized representation
similar_values = {
    'Ts': 'TS',
    'mk': 'MK',
    'Ex': 'EX',
    'Le': 'LE',
    'Np': 'NP',
    'Yo': 'YO',
    'Bt': 'BT',
    'Hg': 'HG',
}

# Apply the mapping to merge similar values
data_mart_df_V3['postcode_prefix'] = data_mart_df_V3['postcode_prefix'].map(similar_values).fillna(data_mart_df_V3['postcode_prefix'])

# Remove duplicate values caused by the mapping
data_mart_df_V3['postcode_prefix'] = data_mart_df_V3['postcode_prefix'].str.upper()  # Convert to uppercase to ensure consistency
data_mart_df_V3['postcode_prefix'] = data_mart_df_V3['postcode_prefix'].drop_duplicates()

# Check unique values after corrections and merging
print(data_mart_df_V3['postcode_prefix'].unique())


['OTHER' nan 'M' 'SN' 'GL' 'LE' 'BA' 'DY' 'GU' 'BS' 'BH' 'SS' 'UB' 'CO'
 'CT' 'RG' 'SA' 'TR' 'B' 'RM' 'NR' 'DT' 'N' 'EX' 'OX' 'E' 'KT' 'SP' 'WF'
 'WS' 'NW' 'CF' 'SL' 'WA' 'L' 'BN' 'CR' 'CH' 'S' 'HU' 'DN' 'TN' 'TA' 'PE'
 'EH' 'BL' 'TD' 'NE' 'NP' 'CV' 'MK' 'ME' 'SO' 'PO' 'NN' 'DE' 'DG' 'LL'
 'CW' 'BD' 'HP' 'BB' 'IP' 'SM' 'TW' 'ML' 'LN' 'TF' 'AB' 'TS' 'CM' 'DA'
 'SW' 'WN' 'EN' 'PA' 'RH' 'SG' 'W' 'FY' 'WD' 'TQ' 'DH' 'NG' 'IG' 'SY' 'WR'
 'HD' 'FK' 'EC' 'PH' 'BR' 'CB' 'LS' 'SK' 'BT' 'HA' 'KY' 'LU' 'G' 'ST' 'PR'
 'WV' 'SE' 'CA' 'DL' 'SR' 'YO' 'OL' 'IV' 'WC' 'DD' 'HR' 'LA' 'PL' 'KA'
 'AL' 'ZE' 'HX' 'KW' 'HG' 'LD' 'HS' 'C' 'H' 'WG' 'A' 'SSO ' 'KN' 'BX' 'IM'
 'P' ' NP' 'HB' ': LE' 'BSL ' 'AD' 'LG' ' SA' ' PO']


In [62]:
data_mart_df_V3.nationality.unique()

array(['Other', 'British', 'German', 'Welsh', 'Israeli', 'South Korean',
       'Dutch', 'Irish', 'Italian', 'Taiwanese', 'Polish', 'Malaysian',
       'Japanese', 'English', 'Austrian', 'Spanish', 'Swiss', 'Danish',
       'American', 'Swedish', 'Belgian', 'Norwegian', 'French', 'Korean',
       'Canadian', 'Indian', 'Lithuanian', 'Northern Irish', 'Australian',
       'Finnish', 'New Zealander', 'Romanian', 'Chinese', 'Scottish',
       'Brtish', 'Russian', 'Saudi Arabian', 'Turkish', 'Palestinian',
       'Emirati', 'German,British', 'Portuguese', 'United Kingdom',
       'Cypriot', 'Latvian', 'Sri Lankan', 'South African', 'Brazilian',
       'Pakistani', 'Zimbabwean', 'Burmese', 'Singaporean', 'Bulgarian',
       'Mauritian', 'Hungarian', 'Czech', 'Greek', 'Ukrainian', 'Kenyan',
       'Albanian,British', 'Guyanese', 'Nepalese', 'Nigerian', 'Albanian',
       'Tanzanian', 'British,Portuguese', 'Bangladeshi', 'British,',
       'Ghanaian', 'British,Italian', 'Jordanian', 'Slovak', 

In [63]:
# Define a dictionary mapping synonymous values to a standardized representation
synonymous_values = {
    'United Kingdom': 'British',
    'Brtish': 'British',
    'Brittish': 'British',
    'English': 'British',
    'Welsh': 'British',
    'Irish': 'British',  # Assuming Irish nationals living in the UK are included as British
    'Northern Irish': 'British',  # Assuming Northern Irish nationals are included as British
    'Cymraes': 'British',  # Assuming this refers to Welsh nationality
    'Scotland': 'British',  # Assuming Scottish nationals living in the UK are included as British
    'England': 'British',  # Assuming English nationals living in the UK are included as British
    'Irish,American': 'American, Irish',  # Assuming dual nationality
    'German,South African': 'German, South African'  # Assuming dual nationality
}

# Apply the mapping to merge similar values
data_mart_df_V3['nationality'] = data_mart_df_V3['nationality'].map(synonymous_values).fillna(data_mart_df_V3['nationality'])


In [64]:
def preprocess_text_similarity(text):
    # Tokenization
    tokens = word_tokenize(text)

    # Lowercasing and remove punctuation and special characters
    tokens = [re.sub(r'[^a-zA-Z0-9]', '', token.lower()) for token in tokens]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Join tokens back into a single string
    processed_text = ' '.join(tokens)

    return processed_text

In [65]:

data_mart_df_V3.section.unique()

array(['Other',
       'WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES',
       'AGRICULTURE, FORESTRY AND FISHING', 'MANUFACTURING',
       'CONSTRUCTION', 'OTHER SERVICE ACTIVITIES',
       'TRANSPORTATION AND STORAGE',
       'HUMAN HEALTH AND SOCIAL WORK ACTIVITIES',
       'ACCOMMODATION AND FOOD SERVICE ACTIVITIES',
       'PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES', 'EDUCATION',
       'ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES',
       'MINING AND QUARRYING', 'REAL ESTATE ACTIVITIES',
       'ELECTRICITY, GAS, STEAM AND AIR CONDITIONING SUPPLY',
       'INFORMATION AND COMMUNICATION',
       'FINANCIAL AND INSURANCE ACTIVITIES',
       'WATER SUPPLY; SEWERAGE, WASTE MANAGEMENT AND REMEDIATION ACTIVITIES',
       'ARTS, ENTERTAINMENT AND RECREATION',
       'ACTIVITIES OF EXTRATERRITORIAL ORGANISATIONS AND BODIES',
       'PUBLIC ADMINISTRATION AND DEFENCE; COMPULSORY SOCIAL SECURITY',
       'ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS; UNDIFFERENTIA

In [66]:
# Apply the preprocessing function to the 'ruc11' column
data_mart_df_V3['processed_ruc11'] = data_mart_df_V3['ruc11'].apply(preprocess_text_similarity)
data_mart_df_V3.processed_ruc11.unique()

array(['', 'urbanwithsignificantrur', 'largelyrur',
       'urbanwithmajorconurb', 'urbanwithcityandtown'], dtype=object)

In [67]:
# Apply the preprocessing function to the 'section' column
data_mart_df_V3['processed_section'] = data_mart_df_V3['section'].apply(preprocess_text_similarity)
data_mart_df_V3.processed_section.unique()

array(['', 'wholesal retail trade  repair motor vehicl motorcycl',
       'agricultur  forestri fish', 'manufactur', 'construct',
       'servic activ', 'transport storag',
       'human health social work activ', 'accommod food servic activ',
       'profession  scientif technic activ', 'educ',
       'administr support servic activ', 'mine quarri',
       'real estat activ', 'electr  ga  steam air condit suppli',
       'inform commun', 'financi insur activ',
       'water suppli  sewerag  wast manag remedi activ',
       'art  entertain recreat', 'activ extraterritori organis bodi',
       'public administr defenc  compulsori social secur',
       'activ household employ  undifferenti goodsand servicesproduc activ household use'],
      dtype=object)

In [68]:
# Apply the preprocessing function to the 'uk_region' column
data_mart_df_V3['processed_uk_region'] = data_mart_df_V3['uk_region'].apply(preprocess_text_similarity)
data_mart_df_V3.processed_uk_region.unique()

array(['', 'northwest', 'southwest', 'eastengland', 'westmidland',
       'southeast', 'greaterlondon', 'wale', 'northeast', 'scotland',
       'northernireland', 'nongeograph', 'isleofman'], dtype=object)

In [69]:

# Apply the preprocessing function to the 'division' column
data_mart_df_V3['processed_division'] = data_mart_df_V3['division'].apply(preprocess_text_similarity)
data_mart_df_V3.processed_division.unique()

array(['', 'wholesal trade  except motor vehicl motorcycl',
       'wholesal retail trade repair motor vehicl motorcycl',
       'crop anim product  hunt relat servic activ',
       'repair instal machineri equip', 'specialis construct activ',
       'manufactur fabric metal product  except machineri equip',
       'construct build', 'person servic activ',
       'wareh support activ transport', 'human health activ', 'accommod',
       'manufactur food product', 'scientif research develop',
       'manufactur basic metal', 'educ', 'activ membership organis',
       'manufactur machineri equip nec ', 'forestri log',
       'offic administr  offic support busi support activ',
       'land transport transport via pipelin', 'residenti care activ',
       'rental leas activ', 'mine coal lignit',
       'architectur engin activ  technic test analysi',
       'real estat activ', 'fish aquacultur',
       'electr  ga  steam air condit suppli', 'manufactur',
       'manufactur wood product wood

In [ ]:
# LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder
data_mart_df_V3.account_source = label_encoder.fit_transform(data_mart_df_V3.account_source)
data_mart_df_V3.siccode_sictext_1 = label_encoder.fit_transform(data_mart_df_V3.siccode_sictext_1)
data_mart_df_V3.nationality = label_encoder.fit_transform(data_mart_df_V3.nationality)
data_mart_df_V3.processed_section = label_encoder.fit_transform(data_mart_df_V3.processed_section)
data_mart_df_V3.processed_division = label_encoder.fit_transform(data_mart_df_V3.processed_division)
data_mart_df_V3.processed_uk_region = label_encoder.fit_transform(data_mart_df_V3.processed_uk_region)
data_mart_df_V3.processed_ruc11 = label_encoder.fit_transform(data_mart_df_V3.processed_ruc11)
data_mart_df_V3.postcode_prefix = label_encoder.fit_transform(data_mart_df_V3.postcode_prefix)


In [71]:
get_dummy_cols = ['domain_type', 'lineofbusiness', 'companycategory', 'uk_region', 'country', 'county', 'ruc11']
# Convert categorical variable into dummy/indicator variables
for col in get_dummy_cols:
  data_mart_df_V3 = pd.concat([data_mart_df_V3, pd.get_dummies(data_mart_df_V3[col],
                                                               prefix=f"{col}_cat")], axis=1)


# Now drop the original categorical columns
data_mart_df_V3.drop(get_dummy_cols, axis=1, inplace=True)

In [72]:
data_mart_df_V3.shape

(79066, 133)

In [73]:
drop_cols = ['postcode_district',
             'account_source', 'siccode_sictext_1', 'nationality',
             'section', 'division', 'postcode_prefix']
# Drop columns from the DataFrame
data_mart_df_V3.drop(columns=drop_cols, inplace=True)

In [74]:
data_mart_df_V3.iloc[:, 80:].head()

,companycategory_cat_Private_Limited_Company,companycategory_cat_Private_Unlimited_Company,companycategory_cat_Public_Limited_Company,uk_region_cat_East_England,uk_region_cat_Greater_London,uk_region_cat_Isle_of_Man,uk_region_cat_Non-geographic,uk_region_cat_North_East,uk_region_cat_North_West,uk_region_cat_Northern_Ireland,uk_region_cat_Other,uk_region_cat_Scotland,uk_region_cat_South_East,uk_region_cat_South_West,uk_region_cat_Wales,uk_region_cat_West_Midlands,country_cat_England,country_cat_Isle_of_Man,country_cat_Other,country_cat_Scotland,country_cat_Wales,county_cat_Other,county_cat_cumbria,county_cat_devon,county_cat_essex,county_cat_gloucestershire,county_cat_hampshire,county_cat_hertfordshire,county_cat_kent,county_cat_lancashire,county_cat_norfolk,county_cat_north_yorkshire,county_cat_nottinghamshire,county_cat_oxfordshire,county_cat_somerset,county_cat_staffordshire,county_cat_suffolk,county_cat_surrey,county_cat_warwickshire,county_cat_west_sussex,county_cat_worcestershire,ruc11_cat_Largely_Rural,ruc11_cat_Other,ruc11_cat_Urban_with_City_and_Town,ruc11_cat_Urban_with_Major_Conurbation,ruc11_cat_Urban_with_Significant_Rural
0,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [75]:
data_mart_df_V3.iloc[:, 80:].info()

<class 'pandas.core.frame.DataFrame'>
Index: 79066 entries, 0 to 101349
Data columns (total 46 columns):
 #   Column                                         Non-Null Count  Dtype
---  ------                                         --------------  -----
 0   companycategory_cat_Private_Limited_Company    79066 non-null  bool 
 1   companycategory_cat_Private_Unlimited_Company  79066 non-null  bool 
 2   companycategory_cat_Public_Limited_Company     79066 non-null  bool 
 3   uk_region_cat_East_England                     79066 non-null  bool 
 4   uk_region_cat_Greater_London                   79066 non-null  bool 
 5   uk_region_cat_Isle_of_Man                      79066 non-null  bool 
 6   uk_region_cat_Non-geographic                   79066 non-null  bool 
 7   uk_region_cat_North_East                       79066 non-null  bool 
 8   uk_region_cat_North_West                       79066 non-null  bool 
 9   uk_region_cat_Northern_Ireland                 79066 non-null  bool 
 10  uk

In [76]:
# Replace boolean values in DataFrame with numeric equivalents.

def replace_boolean_with_numeric(df):

    # Select boolean columns
    boolean_cols = df.select_dtypes(include=bool).columns

    # Iterate over boolean columns and replace False with 0 and True with 1
    for col in boolean_cols:
        df[col] = df[col].astype(int)

    return df

In [77]:
data_mart_df_V4 = replace_boolean_with_numeric(data_mart_df_V3)
data_mart_df_V4.iloc[:, 80:].head()

,companycategory_cat_Private_Limited_Company,companycategory_cat_Private_Unlimited_Company,companycategory_cat_Public_Limited_Company,uk_region_cat_East_England,uk_region_cat_Greater_London,uk_region_cat_Isle_of_Man,uk_region_cat_Non-geographic,uk_region_cat_North_East,uk_region_cat_North_West,uk_region_cat_Northern_Ireland,uk_region_cat_Other,uk_region_cat_Scotland,uk_region_cat_South_East,uk_region_cat_South_West,uk_region_cat_Wales,uk_region_cat_West_Midlands,country_cat_England,country_cat_Isle_of_Man,country_cat_Other,country_cat_Scotland,country_cat_Wales,county_cat_Other,county_cat_cumbria,county_cat_devon,county_cat_essex,county_cat_gloucestershire,county_cat_hampshire,county_cat_hertfordshire,county_cat_kent,county_cat_lancashire,county_cat_norfolk,county_cat_north_yorkshire,county_cat_nottinghamshire,county_cat_oxfordshire,county_cat_somerset,county_cat_staffordshire,county_cat_suffolk,county_cat_surrey,county_cat_warwickshire,county_cat_west_sussex,county_cat_worcestershire,ruc11_cat_Largely_Rural,ruc11_cat_Other,ruc11_cat_Urban_with_City_and_Town,ruc11_cat_Urban_with_Major_Conurbation,ruc11_cat_Urban_with_Significant_Rural
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [78]:
data_mart_df_V4 = reduce_mem_usage(data_mart_df_V4)

Mem. usage decreased to 18.02 Mb (65.2% reduction)


In [79]:
# Rename rural_including_hub_towns_rural_&_rural_related_population_as_%_of_total_population_2011
data_mart_df_V4.rename(columns={'rural_including_hub_towns_rural_&_rural_related_population_as_%_of_total_population_2011': 'rural_population_2011'}, inplace=True)


In [80]:
data_mart_df_V4.iloc[:, 40:80].head()

,total_litres_monthly_avg,non_dd_litres_monthly_avg,dd_litres_monthly_avg,total_transactions_monthly_avg,total_transactions_fuel_monthly_avg,total_transactions_non_visa_monthly_avg,total_transactions_visa_monthly_avg,total_spend_monthly_avg,fuel_spend_monthly_avg,non_fuel_spend_non_visa_monthly_avg,non_fuel_spend_visa_monthly_avg,total_ev_transactions_monthly_avg,total_ev_energy_charge_per_kwh_monthly_avg,total_ev_session_charge_per_minute_monthly_avg,ev_spend_monthly_avg,total_rev_monthly_avg,churned,is_holiday,customer_activity_duration,start_date_diff,last_transaction_date_diff,transaction_diversity,company_type_Council_Charity,company_type_Limited,company_type_Other,company_type_PLC,company_type_Partnership,company_type_Sole_Trader,processed_ruc11,processed_section,processed_uk_region,processed_division,domain_type_cat_Other,domain_type_cat_commercial,domain_type_cat_personal,lineofbusiness_cat_AllStar,lineofbusiness_cat_Other,lineofbusiness_cat_TFC,companycategory_cat_Community_Interest_Company,companycategory_cat_Other
0,38.0,38.0,0.0,1.0,1.0,0.0,0.0,55.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0,0,8401,8443,42,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1
1,66.0,66.0,0.0,2.0,2.0,0.0,0.0,92.0,92.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,0,8422,8438,16,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1
2,9.0,9.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,8406,8438,32,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1
3,164.0,164.0,0.0,3.0,3.0,0.0,0.0,223.0,222.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0,0,8416,8438,22,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1
4,62.0,62.0,0.0,1.0,1.0,0.0,0.0,87.0,87.0,0.0,0.0,0.0,0.0,0.0,0.0,8,0,0,7316,7334,18,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1


In [81]:
# Do not use these columns to train the model
do_not_use_cols = ['total_rev_monthly_avg',
                   'total_spend_monthly_avg',
                   'total_transactions_monthly_avg',
                   'churned']

In [82]:
# Set numeric groups

get_bins_cols = ['num_employees', 'fleet_size_declared', 'number_of_web_users', 'number_of_logins_monthly_avg']
get_quarter_cols = ['credit_terms_requested', 'num_live_cards', 'mortgages_nummortcharges',
                    'total_directors_over_time','total_population_2011', 'rural_population_2011']
get_scaler_cols = ['cash_flow_score', 'credit_limit_requested', 'credit_limit_assigned', 'transaction_diversity',
                   'start_date_diff', 'last_transaction_date_diff', 'customer_activity_duration', 'tenure',
                   'commercialdelphicreditlimit', 'commercialdelphiscore',
                   'fuel_sites_monthly_avg', 'total_fee_charge_monthly_avg', 'number_of_distinct_fees_monthly_avg',
                   'sic_code', 'pence_per_litre', 'ruc11cd',  'tenure_months', 'age',  'average_age_of_directors',
                   'total_litres_monthly_avg', 'non_dd_litres_monthly_avg', 'dd_litres_monthly_avg',
                   'total_transactions_fuel_monthly_avg', 'total_transactions_non_visa_monthly_avg',
                   'total_transactions_visa_monthly_avg', 'fuel_spend_monthly_avg',
                   'non_fuel_spend_non_visa_monthly_avg', 'non_fuel_spend_visa_monthly_avg',
                   'total_ev_transactions_monthly_avg', 'total_ev_energy_charge_per_kwh_monthly_avg',
                   'total_ev_session_charge_per_minute_monthly_avg', 'ev_spend_monthly_avg',
                   'processed_ruc11',	'processed_section',	'processed_uk_region',	'processed_division',
                   'total_rev_monthly_avg', 'total_spend_monthly_avg', 'total_transactions_monthly_avg']
manuel_scaler = ['numberofccjsinlast2years']

In [83]:
# Create bins for 'num_employees'
bins = [data_mart_df_V4['num_employees'].min(),
        0,
        data_mart_df_V4['num_employees'].astype(int).mean(),
        data_mart_df_V4['num_employees'].astype(int).mean() + data_mart_df_V4['num_employees'].astype(int).std(),
        data_mart_df_V4['num_employees'].max()]

# Split dataset acording to the bins
bins_df = pd.cut(data_mart_df_V4['num_employees'], bins=bins)

# Create groups with bins
data_shape = data_mart_df_V4.groupby(bins_df).size().reset_index(name='count')

print("Bins and sizes:")
print(data_shape)

Bins and sizes:
          num_employees  count
0           (-1.0, 0.0]   5093
1        (0.0, 147.569]  64298
2   (147.569, 3382.261]   9234
3  (3382.261, 548143.0]    434


In [84]:
# Create bins for 'fleet_size_declared'
bins = [data_mart_df_V4['fleet_size_declared'].min(),
        data_mart_df_V4['fleet_size_declared'].astype(int).mean(),
        data_mart_df_V4['fleet_size_declared'].astype(int).mean() + data_mart_df_V4['fleet_size_declared'].astype(int).std(),
        data_mart_df_V4['fleet_size_declared'].max()]

# Split dataset acording to the bins
bins_df = pd.cut(data_mart_df_V4['fleet_size_declared'], bins=bins)

# Create groups with bins
data_shape = data_mart_df_V4.groupby(bins_df).size().reset_index(name='count')

print("Bins and sizes:")
print(data_shape)

Bins and sizes:
   fleet_size_declared  count
0        (1.0, 38.894]  23098
1    (38.894, 914.061]  48076
2  (914.061, 200000.0]    152


In [85]:
# Create bins for 'number_of_web_users'
bins = [data_mart_df_V4['number_of_web_users'].min(),
        data_mart_df_V4['number_of_web_users'].astype(int).mean(),
        data_mart_df_V4['number_of_web_users'].astype(int).mean() + data_mart_df_V4['number_of_web_users'].astype(int).std(),
        data_mart_df_V4['number_of_web_users'].max()]

# Split dataset acording to the bins
bins_df = pd.cut(data_mart_df_V4['number_of_web_users'], bins=bins)

# Create groups with bins
data_shape = data_mart_df_V4.groupby(bins_df).size().reset_index(name='count')

print("Bins and sizes:")
print(data_shape)

Bins and sizes:
  number_of_web_users  count
0        (1.0, 2.374]  15334
1     (2.374, 40.341]  14225
2    (40.341, 9496.0]    125


In [86]:
#number_of_logins_monthly_avg
# Create bins for 'number_of_logins_monthly_avg'
bins = [data_mart_df_V4['number_of_logins_monthly_avg'].min(),
        data_mart_df_V4['number_of_logins_monthly_avg'].astype(int).mean(),
        data_mart_df_V4['number_of_logins_monthly_avg'].astype(int).mean() + data_mart_df_V4['number_of_logins_monthly_avg'].astype(int).std(),
        data_mart_df_V4['number_of_logins_monthly_avg'].max()]

# Split dataset acording to the bins
bins_df = pd.cut(data_mart_df_V4['number_of_logins_monthly_avg'], bins=bins)

# Create groups with bins
data_shape = data_mart_df_V4.groupby(bins_df).size().reset_index(name='count')

print("Bins and sizes:")
print(data_shape)

Bins and sizes:
  number_of_logins_monthly_avg  count
0                 (1.0, 2.608]  43533
1               (2.608, 5.255]  34407
2               (5.255, 410.0]   1028


In [87]:
# Replace numeric columns in the dataframe with bin codes based on specified bin ranges.

def replace_columns_with_bins(dataframe, column_list):

    for column_name in column_list:
      print(column_name)
      if column_name in dataframe.columns and pd.api.types.is_numeric_dtype(dataframe[column_name]):
          bin_ranges = [dataframe[column_name].min(), dataframe[column_name].astype(int).mean(),
                        dataframe[column_name].astype(int).mean() + dataframe[column_name].astype(int).std(),
                        dataframe[column_name].max()]
          dataframe[column_name] = pd.cut(dataframe[column_name],
                                        bins=bin_ranges,
                                        include_lowest=True,
                                        duplicates='drop').cat.codes
    return dataframe

In [88]:
data_mart_df_V5 = replace_columns_with_bins(data_mart_df_V4, get_bins_cols)
data_mart_df_V5[get_bins_cols].head()

num_employees
fleet_size_declared
number_of_web_users
number_of_logins_monthly_avg


,num_employees,fleet_size_declared,number_of_web_users,number_of_logins_monthly_avg
0,1,1,0,1
1,0,1,0,0
2,1,1,0,0
3,0,1,0,1
4,0,1,0,0


In [89]:
# Convert a numerical column in a DataFrame to quartiles.
def convert_to_quartiles(dataframe, column_name_list):
    for col in column_name_list:
        quartiles = np.percentile(dataframe[col], [0, 25, 50, 75, 100])
        # Add a small epsilon value to ensure uniqueness
        epsilon = 1e-10
        quartiles += epsilon * np.arange(5)
        dataframe[col] = pd.cut(dataframe[col], bins=quartiles, labels=[0.25, 0.50, 0.75, 1], include_lowest=True)

    return dataframe



In [90]:
data_mart_df_V5[get_quarter_cols].head()

,credit_terms_requested,num_live_cards,mortgages_nummortcharges,total_directors_over_time,total_population_2011,rural_population_2011
0,100,2.0,2.0,4.0,970928.0,47.0
1,100,5.0,2.0,4.0,970928.0,47.0
2,100,2.0,2.0,4.0,970928.0,47.0
3,100,12.0,2.0,4.0,970928.0,47.0
4,100,1.0,2.0,4.0,970928.0,47.0


In [91]:
# Convert 'get_quarter_cols' features to quartiles

data_mart_df_V6 = convert_to_quartiles(data_mart_df_V5, get_quarter_cols)
data_mart_df_V6[get_quarter_cols].head()

,credit_terms_requested,num_live_cards,mortgages_nummortcharges,total_directors_over_time,total_population_2011,rural_population_2011
0,0.75,0.25,0.5,0.5,0.5,0.5
1,0.75,0.75,0.5,0.5,0.5,0.5
2,0.75,0.25,0.5,0.5,0.5,0.5
3,0.75,1.00,0.5,0.5,0.5,0.5
4,0.75,0.25,0.5,0.5,0.5,0.5


In [92]:
data_mart_df_V6.credit_terms_requested.value_counts()

credit_terms_requested
0.75    39333
0.25    36977
0.5      2756
1.0         0
Name: count, dtype: int64

In [93]:
data_mart_df_V6.mortgages_nummortcharges.value_counts()

mortgages_nummortcharges
0.5     40632
0.25    29960
1.0      8474
0.75        0
Name: count, dtype: int64

In [94]:
# Initialize MinMaxScaler
scaler = MinMaxScaler()

In [95]:
def normalize_data(dataframe, columns, scaler):

    normalized_dataframe = dataframe.copy()

    # Check if the specified columns exist in the DataFrame
    existing_columns = [col for col in columns if col in normalized_dataframe.columns]

    if existing_columns:
        # Fit and transform the specified columns using MinMaxScaler
        normalized_dataframe[existing_columns] = scaler.fit_transform(normalized_dataframe[existing_columns])

    return normalized_dataframe

In [96]:
data_mart_df_V7 = normalize_data(data_mart_df_V6, get_scaler_cols, scaler)
data_mart_df_V7[get_scaler_cols].head()

,cash_flow_score,credit_limit_requested,credit_limit_assigned,transaction_diversity,start_date_diff,last_transaction_date_diff,customer_activity_duration,tenure,commercialdelphicreditlimit,commercialdelphiscore,fuel_sites_monthly_avg,total_fee_charge_monthly_avg,number_of_distinct_fees_monthly_avg,sic_code,pence_per_litre,ruc11cd,tenure_months,age,average_age_of_directors,total_litres_monthly_avg,non_dd_litres_monthly_avg,dd_litres_monthly_avg,total_transactions_fuel_monthly_avg,total_transactions_non_visa_monthly_avg,total_transactions_visa_monthly_avg,fuel_spend_monthly_avg,non_fuel_spend_non_visa_monthly_avg,non_fuel_spend_visa_monthly_avg,total_ev_transactions_monthly_avg,total_ev_energy_charge_per_kwh_monthly_avg,total_ev_session_charge_per_minute_monthly_avg,ev_spend_monthly_avg,processed_ruc11,processed_section,processed_uk_region,processed_division,total_rev_monthly_avg,total_spend_monthly_avg,total_transactions_monthly_avg
0,0.0825,0.000000,0.001423,0.333333,0.687933,0.025894,0.685663,0.685749,0.000108,0.753125,0.0,0.858212,0.285714,0.527056,0.000183,0.25,0.684864,0.092715,0.376471,0.000365,0.000409,0.0,0.000023,0.0,0.0,0.000020,0.000185,0.0,0.0,0.004122,0.0,0.056054,0.0,0.0,0.0,0.0,0.000116,0.000020,0.000022
1,0.0825,0.000000,0.001423,0.333333,0.687525,0.009864,0.687378,0.687709,0.000125,0.820000,0.0,0.858174,0.142857,0.527056,0.000180,0.25,0.687345,0.092715,0.376471,0.000380,0.000425,0.0,0.000046,0.0,0.0,0.000034,0.000185,0.0,0.0,0.004122,0.0,0.056054,0.0,0.0,0.0,0.0,0.000087,0.000034,0.000044
2,0.0825,0.000071,0.001423,0.333333,0.687525,0.019729,0.686071,0.686403,0.022419,0.650000,0.0,0.858100,0.142857,0.527056,0.000184,0.25,0.684864,0.092715,0.376471,0.000350,0.000392,0.0,0.000000,0.0,0.0,0.000004,0.000185,0.0,0.0,0.004122,0.0,0.056054,0.0,0.0,0.0,0.0,0.000029,0.000004,0.000000
3,0.0825,0.000000,0.001423,0.333333,0.687525,0.013564,0.686888,0.687056,0.000091,0.541250,0.0,0.858406,0.000000,0.527056,0.000179,0.25,0.687345,0.092715,0.376471,0.000430,0.000482,0.0,0.000069,0.0,0.0,0.000081,0.000185,0.0,0.0,0.004122,0.0,0.056054,0.0,0.0,0.0,0.0,0.000116,0.000081,0.000066
4,0.0825,0.000000,0.001423,0.333333,0.597455,0.011097,0.597077,0.597223,0.000876,0.833125,0.0,0.858304,0.571429,0.527056,0.000184,0.25,0.598015,0.092715,0.376471,0.000378,0.000423,0.0,0.000023,0.0,0.0,0.000032,0.000185,0.0,0.0,0.004122,0.0,0.056054,0.0,0.0,0.0,0.0,0.000231,0.000032,0.000022


In [97]:
data_mart_df_V7.numberofccjsinlast2years.value_counts()

numberofccjsinlast2years
0.000000    71393
1.000000      538
0.500000      385
0.333252      311
0.250000      243
            ...  
0.685547        1
2.916016        1
1.812500        1
2.476562        1
0.264648        1
Name: count, Length: 602, dtype: int64

In [98]:
# Create bins for 'numberofccjsinlast2years'
bins = [-1,
        data_mart_df_V7['numberofccjsinlast2years'].min(),
        data_mart_df_V7['numberofccjsinlast2years'].astype(int).mean(),
        data_mart_df_V7['numberofccjsinlast2years'].astype(int).mean() + data_mart_df_V7['numberofccjsinlast2years'].astype(int).std(),
        data_mart_df_V7['numberofccjsinlast2years'].max()]

# Split dataset acording to the bins
bins_df = pd.cut(data_mart_df_V7['numberofccjsinlast2years'], bins=bins)

# Create groups with bins
data_shape = data_mart_df_V7.groupby(bins_df).size().reset_index(name='count')

print("Bins and sizes:")
print(data_shape)

Bins and sizes:
  numberofccjsinlast2years  count
0              (-1.0, 0.0]  71393
1            (0.0, 0.0365]    105
2          (0.0365, 0.794]   5735
3         (0.794, 133.625]   1833


In [99]:
numberofccjsinlast2years_bin_ranges = [data_mart_df_V7['numberofccjsinlast2years'].min(),
                                       data_mart_df_V7['numberofccjsinlast2years'].astype(int).mean(),
                                       data_mart_df_V7['numberofccjsinlast2years'].astype(int).mean() + data_mart_df_V7['numberofccjsinlast2years'].astype(int).std(),
                                       data_mart_df_V7['numberofccjsinlast2years'].max()]
data_mart_df_V7['numberofccjsinlast2years'] = pd.cut(data_mart_df_V7['numberofccjsinlast2years'],
                              bins=numberofccjsinlast2years_bin_ranges,
                              include_lowest=True,
                              duplicates='drop').cat.codes
data_mart_df_V7['numberofccjsinlast2years'].head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: numberofccjsinlast2years, dtype: int8

In [100]:
data_mart_df_V7['numberofccjsinlast2years'].value_counts()

numberofccjsinlast2years
0    71498
1     5735
2     1833
Name: count, dtype: int64

In [103]:
do_not_use_cols

['total_rev_monthly_avg',
 'total_spend_monthly_avg',
 'total_transactions_monthly_avg',
 'churned']

In [104]:
data_mart_df_V7.last_transaction_date.max()

Timestamp('2024-03-15 00:00:00')

In [106]:
data_mart_df_V7[data_mart_df_V7.churned == 1].last_transaction_date.max()

Timestamp('2023-12-17 00:00:00')

In [107]:
data_mart_df_V7[data_mart_df_V7.start_date < '2023-10-15'].shape, data_mart_df_V7[data_mart_df_V7.start_date >= '2023-10-15'].shape


((75915, 126), (3151, 126))

In [108]:
data_mart_df_V7[data_mart_df_V7.start_date >= '2023-10-15'].churned.value_counts()

churned
0    2974
1     177
Name: count, dtype: int64

In [109]:
train_df = data_mart_df_V7[data_mart_df_V7.start_date < '2023-10-15']
test_df = data_mart_df_V7[data_mart_df_V7.start_date >= '2023-10-15']
train_df.shape, test_df.shape

((75915, 126), (3151, 126))

((75915, 126), (3151, 126))

In [505]:
result_df = test_df[['toplevelcustomerid', 'client_account_number',
                     'start_date', 'last_transaction_date']]
result_df.shape

(3151, 4)

In [111]:
# Read dataset

%%time
result_df_path = 'churn_results.csv'

try:
    result_df = pd.read_csv(result_df_path)
except pd.errors.ParserError as e:
    print(f'Error while parsing CSV file: {e}')



result_df.shape

CPU times: user 12.4 ms, sys: 2.19 ms, total: 14.6 ms
Wall time: 54.9 ms


(3151, 10)

In [112]:
result_df.shape, data_mart_df.shape

((3151, 10), (79066, 104))

In [507]:
# Merge dataframes on 'toplevelcustomerid' column
result_df = pd.merge(result_df, data_mart_df[['toplevelcustomerid', 'total_rev_monthly_avg',
                                              'total_spend_monthly_avg', 'total_transactions_monthly_avg', 'churned']],
                     on='toplevelcustomerid', how='left')
result_df.shape

(3151, 8)

In [508]:
result_df.head()

,toplevelcustomerid,client_account_number,start_date,last_transaction_date,total_rev_monthly_avg,total_spend_monthly_avg,total_transactions_monthly_avg,churned
0,230845,AS60222357,2023-10-16,2023-12-30,35,382.0,4.0,0
1,230846,AS60222358,2023-10-16,2024-01-15,4,171.0,2.0,0
2,230847,AS60222359,2023-10-16,2024-02-27,15,1111.0,14.0,0
3,230848,AS60222360,2023-10-16,2024-03-12,10,769.0,10.0,0
4,230851,AS60222363,2023-10-16,2024-02-23,20,1536.0,22.0,0


In [113]:
train_df.drop(columns=['toplevelcustomerid', 'client_account_number',
                       'start_date', 'last_transaction_date'], inplace=True)
test_df.drop(columns=['toplevelcustomerid', 'client_account_number',
                     'start_date', 'last_transaction_date'], inplace=True)

<ipython-input-113-ba5ea4cf9ec9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(columns=['toplevelcustomerid', 'client_account_number',
<ipython-input-113-ba5ea4cf9ec9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(columns=['toplevelcustomerid', 'client_account_number',


In [114]:
# Defining a method to move target data to the end of the dataframe
def move_target_to_end(df, target_column):

    # Ensure the target column is in the DataFrame
    if target_column not in df.columns:
        print(f"Error: Target column '{target_column}' not found in the DataFrame.")
        return df

    # Reorder columns to move the target column to the end
    new_order = [col for col in df.columns if col != target_column] + [target_column]
    df = df[new_order]

    return df

In [115]:
# Move target column to the end of list
train_df = move_target_to_end(train_df, 'churned')
test_df = move_target_to_end(test_df, 'churned')

In [116]:
# Split train and test datas for input and output
y_train = train_df['churned'].copy() #features (independent variables)
X_train = train_df.drop(columns=['churned']) #features (independent variables)
y_test = test_df['churned'].copy() #features (independent variables)
X_test = test_df.drop(columns=['churned']) #features (independent variables)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((75915, 121), (75915,), (3151, 121), (3151,))

In [117]:
X_train = reduce_mem_usage(X_train)
X_test = reduce_mem_usage(X_test)

Mem. usage decreased to 12.38 Mb (57.8% reduction)
Mem. usage decreased to  0.52 Mb (57.7% reduction)


In [514]:
result_df.to_csv("churn_results.csv", index=False)

result_df.head()

,toplevelcustomerid,client_account_number,start_date,last_transaction_date,total_rev_monthly_avg,total_spend_monthly_avg,total_transactions_monthly_avg,churned
0,230845,AS60222357,2023-10-16,2023-12-30,35,382.0,4.0,0
1,230846,AS60222358,2023-10-16,2024-01-15,4,171.0,2.0,0
2,230847,AS60222359,2023-10-16,2024-02-27,15,1111.0,14.0,0
3,230848,AS60222360,2023-10-16,2024-03-12,10,769.0,10.0,0
4,230851,AS60222363,2023-10-16,2024-02-23,20,1536.0,22.0,0


In [118]:
result_df.churned.value_counts()

churned
0    2974
1     177
Name: count, dtype: int64

In [119]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((75915, 121), (75915,), (3151, 121), (3151,))

In [120]:
y_train.value_counts()

churned
0    41058
1    34857
Name: count, dtype: int64

##Naive Bias

In [519]:
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score, classification_report
from sklearn.cluster import MeanShift, estimate_bandwidth



In [520]:
%%time
# Uncalibrated Naive Bias Model
clf_nb = GaussianNB()
clf_nb.fit(X_train, y_train)
y_test_predict_nb = clf_nb.predict_proba(X_test)[:, 1]
yhat_nb = clf_nb.predict(X_test)
fraction_of_positives_nb, mean_predicted_value_nb = calibration_curve(y_test, y_test_predict_nb, n_bins=10)

CPU times: user 254 ms, sys: 127 ms, total: 381 ms
Wall time: 388 ms


In [521]:
print(classification_report(y_test, yhat_nb))


              precision    recall  f1-score   support

           0       1.00      0.87      0.93      2974
           1       0.32      0.99      0.48       177

    accuracy                           0.88      3151
   macro avg       0.66      0.93      0.71      3151
weighted avg       0.96      0.88      0.91      3151



In [523]:
result_df['churn_prob'] = y_test_predict_nb
result_df.head()

,toplevelcustomerid,client_account_number,start_date,last_transaction_date,total_rev_monthly_avg,total_spend_monthly_avg,total_transactions_monthly_avg,churned,churn_prob
0,230845,AS60222357,2023-10-16,2023-12-30,35,382.0,4.0,0,1.0
1,230846,AS60222358,2023-10-16,2024-01-15,4,171.0,2.0,0,0.0
2,230847,AS60222359,2023-10-16,2024-02-27,15,1111.0,14.0,0,0.0
3,230848,AS60222360,2023-10-16,2024-03-12,10,769.0,10.0,0,0.0
4,230851,AS60222363,2023-10-16,2024-02-23,20,1536.0,22.0,0,0.0


In [524]:
result_df.churn_prob.value_counts()

churn_prob
0.0    2596
1.0     553
1.0       1
1.0       1
Name: count, dtype: int64

In [525]:
result_df.to_csv("churn_results.csv", index=False)


## KNN Classifier

In [532]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier



In [533]:
# Define the parameters grid for Grid Search
param_grid = {
    'n_neighbors': [3, 5, 7],  # Number of neighbors to consider
    'weights': ['uniform', 'distance'],  # Weighting function used in prediction
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']  # Algorithm to compute nearest neighbors
}


In [534]:
%%time
# Create a KNN classifier
knn = KNeighborsClassifier()

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the Grid Search to the data
grid_search.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


CPU times: user 1min 27s, sys: 18.4 s, total: 1min 45s
Wall time: 2h 53min 46s


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [3, 5, 7],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [535]:

# Get the best parameters found by Grid Search
best_params = grid_search.best_params_

# Create a new KNN classifier with the best parameters
best_knn = KNeighborsClassifier(**best_params)


In [536]:
%%time
# Fit the best classifier to the training data
best_knn.fit(X_train, y_train)

# Predict probabilities for the test data
y_test_predict_knn = best_knn.predict_proba(X_test)[:, 1]
yhat_knn = best_knn.predict(X_test)


# The y_probs will contain the probability estimates for each class

fraction_of_positives, mean_predicted_value = calibration_curve(y_test, y_test_predict_knn, n_bins=10)


CPU times: user 9.91 s, sys: 142 ms, total: 10.1 s
Wall time: 6.8 s


In [538]:
print(classification_report(y_test, yhat_knn))

              precision    recall  f1-score   support

           0       1.00      0.85      0.92      2974
           1       0.28      0.97      0.43       177

    accuracy                           0.86      3151
   macro avg       0.64      0.91      0.67      3151
weighted avg       0.96      0.86      0.89      3151



In [539]:
result_df['churn_prob_knn'] = y_test_predict_knn
result_df.head()

,toplevelcustomerid,client_account_number,start_date,last_transaction_date,total_rev_monthly_avg,total_spend_monthly_avg,total_transactions_monthly_avg,churned,churn_prob,churn_prob_knn
0,230845,AS60222357,2023-10-16,2023-12-30,35,382.0,4.0,0,1.0,1.0
1,230846,AS60222358,2023-10-16,2024-01-15,4,171.0,2.0,0,0.0,0.0
2,230847,AS60222359,2023-10-16,2024-02-27,15,1111.0,14.0,0,0.0,0.0
3,230848,AS60222360,2023-10-16,2024-03-12,10,769.0,10.0,0,0.0,0.0
4,230851,AS60222363,2023-10-16,2024-02-23,20,1536.0,22.0,0,0.0,0.0


In [540]:
result_df.churn_prob_knn.value_counts()

churn_prob_knn
0.000000    2327
1.000000     439
0.797043       1
0.801854       1
0.210523       1
            ... 
0.251437       1
0.191070       1
0.396408       1
0.192026       1
0.612531       1
Name: count, Length: 387, dtype: int64

In [541]:
result_df.to_csv("churn_results.csv", index=False)


## Random Forest

In [132]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score, classification_report




In [122]:
# Define parameters
n_estimators = [700]
max_depth = [10, 8]
min_samples_split = [10, 5]
min_samples_leaf = [5, 3]

In [123]:
# Grid Search components
hyper_random = {"n_estimators":n_estimators,
                "max_depth":max_depth,
                "min_samples_split":min_samples_split,
                "min_samples_leaf":min_samples_leaf}

In [124]:
%%time
#Create GridSearch Pipeline
clf_rf_tuned = GridSearchCV(RandomForestClassifier(), hyper_random,
                            cv = 5, verbose = 1,
                            n_jobs = -1)
clf_rf_tuned.fit(X_train, y_train)


Fitting 5 folds for each of 8 candidates, totalling 40 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


CPU times: user 1min 18s, sys: 3.63 s, total: 1min 21s
Wall time: 27min 9s


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [10, 8], 'min_samples_leaf': [5, 3],
                         'min_samples_split': [10, 5], 'n_estimators': [700]},
             verbose=1)

In [125]:
best_params_random = clf_rf_tuned.best_params_
print(best_params_random)

{'max_depth': 8, 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 700}


In [126]:
CV_clf_rf = RandomForestClassifier(max_depth=best_params_random["max_depth"],
                                   min_samples_leaf=best_params_random["min_samples_leaf"],
                                   min_samples_split=best_params_random["min_samples_split"],
                                   n_estimators= best_params_random["n_estimators"])

In [130]:
%%time
# Fit model
CV_clf_rf.fit(X_train, y_train)
y_test_predict_random = CV_clf_rf.predict_proba(X_test)[:, 1]
yhat_random = CV_clf_rf.predict(X_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_test, y_test_predict_random, n_bins=10)

CPU times: user 1min 5s, sys: 121 ms, total: 1min 5s
Wall time: 1min 10s


In [133]:
print(classification_report(y_test, yhat_random))


              precision    recall  f1-score   support

           0       1.00      0.99      1.00      2974
           1       0.91      1.00      0.95       177

    accuracy                           0.99      3151
   macro avg       0.95      1.00      0.97      3151
weighted avg       0.99      0.99      0.99      3151



In [134]:
result_df['churn_prob_rf'] = y_test_predict_random
result_df.head()

,toplevelcustomerid,client_account_number,start_date,last_transaction_date,total_rev_monthly_avg,total_spend_monthly_avg,total_transactions_monthly_avg,churned,churn_prob,churn_prob_knn,churn_prob_rf
0,230845,AS60222357,2023-10-16,2023-12-30,35,382.0,4.0,0,1.0,1.0,0.472028
1,230846,AS60222358,2023-10-16,2024-01-15,4,171.0,2.0,0,0.0,0.0,0.179686
2,230847,AS60222359,2023-10-16,2024-02-27,15,1111.0,14.0,0,0.0,0.0,0.116830
3,230848,AS60222360,2023-10-16,2024-03-12,10,769.0,10.0,0,0.0,0.0,0.050564
4,230851,AS60222363,2023-10-16,2024-02-23,20,1536.0,22.0,0,0.0,0.0,0.119955


In [135]:
result_df.to_csv("churn_results.csv", index=False)
